In [1]:
import pandas as pd 
import numpy as np
from  matched import matched_business_change, matched_extraction,determine_filed_check

Load Data

In [2]:
IEPO_GROUND_data=pd.read_excel("IEPO_210_Documents.xlsx",sheet_name="IEPO_Cleaned")
IEPO_extracted_data=pd.read_csv("KA_Dump.csv")

In [3]:
doc_ids=IEPO_GROUND_data["document_id"].unique()

IEPO_extracted_data = IEPO_extracted_data[
    IEPO_extracted_data["document_id"].isin(doc_ids)
]


In [4]:
IEPO_extracted_data.loc[:, 'running_index'] =IEPO_extracted_data.groupby('document_id').cumcount() + 1
IEPO_extracted_data = IEPO_extracted_data.reset_index(drop=True)

#adjust the name of columns
IEPO_GROUND_data = IEPO_GROUND_data.rename(columns={"gt_qty": "gt_quantity"})
#IEPO_GROUND_data =IEPO_GROUND_data.drop_duplicates()

In [5]:
IEPO_GROUND_data =IEPO_GROUND_data.drop_duplicates()

In [7]:

#gather data with changes 
QTY_extraction=matched_extraction(IEPO_GROUND_data, IEPO_extracted_data, "quantity")
QTY_business_change=matched_business_change(IEPO_GROUND_data, QTY_extraction, "quantity")
unit_extraction=matched_extraction(IEPO_GROUND_data, QTY_extraction, "unit")
unit_business_change=matched_business_change(IEPO_GROUND_data, unit_extraction, "unit")

In [8]:
Final_data=unit_business_change

In [13]:

Final_data["quantity_check"] = Final_data.apply(lambda row: determine_filed_check(row, field="quantity"), axis=1)

Final_data["unit_check"] = Final_data.apply(lambda row: determine_filed_check(row, field="unit"), axis=1)

In [18]:
Final_data["Delete__qty_status"] = "Alternatives"

# Apply conditions
Final_data.loc[
    (Final_data["match_corrected_quantity"] == "position_mismatch,no change") & 
    (Final_data["quantity_status"] == "DELETED"),
    "Delete__qty_status"
] = "Position Misalignment"

Final_data.loc[
    ((Final_data["quantity_original"].isna())  & (Final_data["quantity_corrected"].isna())) & 
    (Final_data["quantity_status"] == "DELETED"),
    "Delete__qty_status"
] = "Null Value Issue"        

In [20]:
Final_data["Delete__unit_status"] = "Alternatives"

# Apply conditions
Final_data.loc[
    (Final_data["match_corrected_quantity"] == "position_mismatch,no change") & 
    (Final_data["quantity_status"] == "DELETED"),
    "Delete__unit_status"
] = "Position Misalignment"

Final_data.loc[
    ((Final_data["quantity_original"].isna())  & (Final_data["quantity_corrected"].isna())) & 
    (Final_data["quantity_status"] == "DELETED"),
    "Delete__unit_status"
] = "Null Value Issue" 